In [ ]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']= '3'

from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.densenet import preprocess_input 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

from generator_v1 import BalencedDataGenerator, remove_padding, crop_or_pad
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

# 

## Model & Data Load

In [ ]:
model = load_model('./logs/epoch-18-0.99317.h5')
model.summary()

In [ ]:
data_df = pd.read_csv('./dataset.csv',index_col=0)
test_df = data_df.loc[data_df['is_train']==False].drop(['is_train'],axis=1).reset_index(drop=True)

# 

## Model Evaluation - Confusion Matrix

In [ ]:
import math

In [ ]:
# Test Data Load

data_config ={
    'batch_size' : 32,
    'num_class' : 4,
    'img_h' : 224,
    'img_w' : 224
}

In [ ]:
n_datas = test_df.shape[0]
n_batch_epoch = int(n_datas/data_config['batch_size'])

y_pred = []
y_label = []

for i in tqdm(range(n_batch_epoch)):
    image_batch = np.zeros((data_config['batch_size'],224,224,3),np.uint8)    
    
    for i,(img_path,label) in enumerate(test_df[["image","label"]][i*data_config['batch_size']:(i+1)*data_config['batch_size']].values):
        image = cv2.imread(img_path)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = remove_padding(image)
        image = crop_or_pad(image, [224,224])

        y_label.append(label)

        image_batch[i,:,:] = image
        
    result = model.predict(image_batch)
    y_pred += list(np.argmax(result, axis=-1))

In [ ]:
# draw confusion matrix
class_names =['0', '90', '180', '270']


cm = confusion_matrix(y_label, y_pred)
cm_draw = ConfusionMatrixDisplay(cm, display_labels=class_names)

cm_draw.plot(cmap=plt.cm.Blues)

print(classification_report(y_label, y_pred, target_names=class_names))
plt.savefig('cm.png',dpi=300)

# 

## Model Prediction

In [ ]:
idx = 3000

image = cv2.imread(test_df["image"][idx])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

label = test_df["label"][idx]

image = remove_padding(image)
image = crop_or_pad(image, [224,224])

In [ ]:
plt.figure()
plt.imshow(image, cmap=None)
plt.show()

In [ ]:
image = image.reshape(1,224,224,3)
res = model.predict(image)
print(np.argmax(res), label)